# Fire Evacuation Planning for the MI Building

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from vae_utils import *
from vae import VAE

In [ ]:
train_set = np.load("data/FireEvac_train_set.npy")
test_set = np.load("data/FireEvac_test_set.npy")
train_set.shape, test_set.shape

Scatter plot train and test data

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,9))
fig.suptitle("MI BUILDING DATASET")
ax[0].scatter(train_set[:,0], train_set[:,1])
ax[0].set_title("MI BUILDING - train")
ax[1].scatter(test_set[:,0], test_set[:,1], c='red')
ax[1].set_title("MI BUILDING - test")
# Create a Rectangle patch
rect = patches.Rectangle((130, 50), 20, 20, linewidth=1, edgecolor='r', facecolor='none')
# Add the patch to the Axes
ax[0].add_patch(rect)
plt.show()

Scale between -1 and 1

In [ ]:
minmax_scaler = MinMaxScaler(feature_range=(-1,1))  # scaling important also to later rescale back
train_set = minmax_scaler.fit_transform(train_set)
test_set = minmax_scaler.transform(test_set)

fig, ax = plt.subplots(1,2, figsize=(15,9))
fig.suptitle("MI BUILDING DATASET")
ax[0].scatter(train_set[:,0], train_set[:,1])
ax[0].set_title("MI BUILDING - train")
ax[1].scatter(test_set[:,0], test_set[:,1], c='red')
ax[1].set_title("MI BUILDING - test")
plt.show()

In [ ]:
# set learning parameters
epochs = 100
batch_size = 64
lr = 0.001

# prepare dataloaders for both data, useful in pytorch
train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True
)
test_loader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False
)

Train a VAE model on the train data of MI Building

In [ ]:
# prepare model and optimizer
model = VAE(in_features=2, latent_dim=10, intermediate_dim=512)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
train_loss = []
test_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss= fit(model, train_loader, optimizer, train_set, labelled=False)
    train_loss.append(train_epoch_loss)
    print(f"Train Loss: {train_epoch_loss[0]:.4f}, {train_epoch_loss[1]:.4f}, {train_epoch_loss[2]:.4f}")
    test_epoch_loss= test(model, test_loader, test_data=test_set, epoch=epoch)
    test_loss.append(test_epoch_loss)
    print(f"Test Loss: {test_epoch_loss[2]:.4f}")
plot_loss(test_loss=test_loss, epochs=epochs)

Plot both the test (left) and train (right) reconstructions

In [ ]:
# get reconstructions
reconstruction_test = get_MI_reconstruction(model, test_loader, test_set)
reconstruction_train = get_MI_reconstruction(model, train_loader, train_set)

fig, axs = plt.subplots(1,2, figsize=(20,10))

for batch in reconstruction_test:
    batch = minmax_scaler.inverse_transform(batch.detach())
    axs[0].scatter(batch[:,0], batch[:,1])
axs[0].set_xlim(0,200)
axs[0].set_ylim(0,115)

for batch in reconstruction_train:
    batch = minmax_scaler.inverse_transform(batch.detach())
    axs[1].scatter(batch[:,0], batch[:,1])
axs[1].set_xlim(0,200)
axs[1].set_ylim(0,115)
plt.show()

Generate 1000 samples, count how many are inside the box

In [ ]:
num_samples = 1000
mu_rec = model.generate_many(num_samples)
mu_rec = minmax_scaler.inverse_transform(mu_rec.detach())
fig, ax = plt.subplots(1,1)
ax.scatter(mu_rec[:,0], mu_rec[:,1])
rect = patches.Rectangle((130, 50), 20, 20, linewidth=1, edgecolor='r', facecolor='none')
# Add the patch to the Axes
ax.add_patch(rect)
ax.set_xlim(0,200)
ax.set_ylim(0,115)
plt.show()
print(f"In the box there are {check_number_in_box(mu_rec, [130,70], [150,50])} people")